### Mounting Drive

In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Path For Model and Data

In [116]:
outputdir='/content/drive/MyDrive/'
inputpath=outputdir+'dummytest.csv'
outputmodelpath=outputdir+'model.pth'

### Dependencies

In [117]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

#For decision tree scikit learn
from sklearn.ensemble import RandomForestClassifier as forest

#for Neural net
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder as le
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold as SKF



## Loading Model

In [118]:
#loading encodings
le1=le()
le1.classes_=np.load(outputdir+'encodings.npy',allow_pickle=True)

In [119]:
#combining all models to form a final model
class final_model(nn.Module):
  def __init__(self):
        super().__init__()
        self.model1=torch.load(outputdir+'model0.pth')
        self.model1.eval()
        self.model2=torch.load(outputdir+'model1.pth')
        self.model2.eval()
        self.model3=torch.load(outputdir+'model2.pth')
        self.model3.eval()
        self.model4=torch.load(outputdir+'model3.pth')
        self.model4.eval()
        self.model5=torch.load(outputdir+'model4.pth')
        self.model5.eval()
        self
  def forward(self,X):
    with torch.no_grad():
      output=(self.model1(X)+self.model2(X)+self.model3(X)+self.model4(X)+self.model5(X))/5
      output=softmax(output).argmax(axis=1)
      k=np.array(le1.inverse_transform(output))

      return output,k

In [120]:
model=torch.load(outputmodelpath)

In [121]:
def accuracy(ytest,ypred):
  return sum(ypred==ytest)/len(ytest)

## Final Function

In [122]:
def final(fpath):
  testset=pd.read_csv(fpath)
  testset=testset.drop('Unnamed: 0',axis=1)#assuming that the dataset will be exactly same without class columns
  testset=testset.reset_index()#just in case index are not same
  testset=testset.drop('index',axis=1)
  dataf=torch.tensor(np.array(testset),dtype=torch.float32)
  ypred,labelpred=model(dataf)
  return labelpred

  

In [124]:
output=final(inputpath)